<a href="https://colab.research.google.com/github/JoeOlang/NLP/blob/main/Text%20Classification/Swahili/ZindiNews/v1.3_LinearSVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import re, string
import nltk
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV


In [2]:
# Introduce some common Swahili stop words
stop_words_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/swahili_stopwords.csv')
stop_words = stop_words_df['StopWords'].tolist()

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Swahili/Train.csv')
df.head()

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa


In [4]:
df['category'].unique()

array(['Kitaifa', 'Biashara', 'michezo', 'Kimataifa', 'Burudani'],
      dtype=object)

---

In [5]:
labels = df['category']
text = df['content']

X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state=0, test_size=0.3)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_transformed = tf_transformer.transform(X_train_counts)

X_test_counts = count_vect.transform(X_test)
X_test_transformed = tf_transformer.transform(X_test_counts)

labels = LabelEncoder()
y_train_labels_fit = labels.fit(y_train)
y_train_lables_trf = labels.transform(y_train)

print(labels.classes_)

['Biashara' 'Burudani' 'Kimataifa' 'Kitaifa' 'michezo']


In [6]:
linear_svc = LinearSVC()
clf = linear_svc.fit(X_train_transformed,y_train_lables_trf)

calibrated_svc = CalibratedClassifierCV(base_estimator=linear_svc,
                                        cv="prefit")

calibrated_svc.fit(X_train_transformed,y_train_lables_trf)
predicted = calibrated_svc.predict(X_test_transformed)

In [7]:
to_predict = ["KOCHA wa Senegal, Alliou Cisse amesema timu yake itazoea mechi hadi mechi kwenye michuano ya Afcon akiwa na lengo la kufanya vizuri kuliko michuano ya mwisho iliyofanyika Gabon mwaka 2017, ambapo Senegal iliishia robo fainali.Cisse ni miongoni mwa makocha wachache wanaoonekana kwenye michuano hiyo mfululizo kuanzia ile iliyopita. Simba wa Teranga wamepangwa kundi C pamoja na Algeria, Kenya na Tanzania.Akizungumzia nafasi yake kwenye kundi hilo, Cisse alisema kundi liko sawa. “Ni kundi lililo sawa kwa timu yangu, kwa sasa hakuna timu ndogo au kubwa. Timu zote zilizofuzu zitakuwa Misri kupambana na kujaribu kupata matokeo mazuri.” “Tutakuwa na Algeria sio ngeni kwenye michuano hii, pia kuna Kenya na Tanzania. Hatuzidharau Kenya na Tanzania kutokana na hali zao, tunatakiwa kujiandaa vizuri kushindana.”Mechi yake na Algeria itakuwa ya tatu mfululizo, amejiandaa vipi? “ Ni mara ya tatu tunacheza na Algeria kwenye hatua ya makundi. Tulicheza 2015 (Equatorial Guinea), 2017 (Gabon), na sasa Misri, ni historia ndefu kwa timu hizi Algeria imebadilika tangu kuwasili kwa Djamel Belmadi, ambaye ni rafiki yangu, tumekuwa pamoja majirani Paris itakuwa mechi nzuri.”Senegal na Algeria ndio zinapewa nafasi kubwa kwenye kundi hili, lakini Kenya na Tanzania hazipaswi kudharauliwa. “Naweza kusema kwamba Senegal haiiogopi timu yoyote.” Akizungumzia malengo ya timu yake alisema wataichukulia kila mechi kwa uzito wake, kwa sababu wanajua lolote linaweza kutokea. “Ni kweli ni miongoni mwa tunaopewa nafasi kubwa, lakini kufikia kiwango hicho tunatakiwa kumfunga kila tunayekutana naye, na itaanza na Tanzania Juni 23, tunapaswa kufikiria uwezo wetu na si kuwafikia adui zetu."]
p_count = count_vect.transform(to_predict)
p_tfidf = tf_transformer.transform(p_count)
print('Average accuracy on test set={}'.format(np.mean(predicted == labels.transform(y_test))))
print('Predicted probabilities of demo input string are')
print(calibrated_svc.predict_proba(p_tfidf))

Average accuracy on test set=0.8635187580853816
Predicted probabilities of demo input string are
[[4.72068163e-07 4.78728391e-04 1.25123986e-04 1.81042393e-04
  9.99214633e-01]]


In [8]:
dftest = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Swahili/Test.csv')
dftest = dftest.reindex(dftest.columns.tolist() + ['Kitaifa', 'Biashara', 'michezo', 'Kimataifa', 'Burudani'], axis=1)

In [27]:
dftest

,swahili_id,content,Kitaifa,Biashara,michezo,Kimataifa,Burudani
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,"MKUU wa Wilaya ya Bahi, Mkoani Dodoma, Mwanah...",1,2,3,4,5
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,"MWISHONI mwa wiki hii, Timu ya Soka ya Taifa,...",1,2,3,4,5
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,THAMANI ya mauzo ya bidhaa za Afrika Masharik...,1,2,3,4,5
3,00868eeee349e286303706ef0ffd851f39708d37,MENEJA Mawasiliano na Utetezi wa asasi ya AGP...,1,2,3,4,5
4,00a5cb12d3058dcf2e42f277eee599992db32412,"WAZIRI wa Kilimo, Japhet Hasunga amesema seri...",1,2,3,4,5
...,...,...,...,...,...,...,...
1283,feb4b0ae88524c9cee3e50f2301d84a235f3c607,MKURUGENZI Msaidizi Msajili wa Asasi za kirai...,1,2,3,4,5
1284,fecf3f14f47237e02721ed4baa4eb6c11abd239f,Kilomoni alisema jana kesho atazungumzia sual...,1,2,3,4,5
1285,feed09e13586d12139cd59bc20996a4d29706606,"TIMU ya taifa ya soka ya Tanzania Bara, Kilim...",1,2,3,4,5
1286,ffc0bca6fde8fa0cce1ac3a7b0d746603d441bf8,"YANGA imetozwa faini ya Sh 3,500,000 kutokana...",1,2,3,4,5


In [21]:
def predict(content):
    #content = [string_text]
    p_count = count_vect.transform(content)
    p_tfidf = tf_transformer.transform(p_count)
    predictions = calibrated_svc.predict_proba(p_tfidf)
    #predictions = predictions.flatten()
    #scores = predictions.tolist()

    return predictions

In [22]:
x = predict(to_predict)

In [23]:
x

array([[4.72068163e-07, 4.78728391e-04, 1.25123986e-04, 1.81042393e-04,
        9.99214633e-01]])

In [24]:
q, r, s, t, w = x.shape

ValueError: ignored

In [25]:
def rt():
    return 1, 2, 3, 4, 5

In [26]:
dftest['Kitaifa'], dftest['Biashara'], dftest['michezo'], dftest['Kimataifa'], dftest['Burudani']   = rt()

In [12]:
dftest['Kitaifa'], dftest['Biashara'], dftest['michezo'], dftest['Kimataifa'], dftest['Burudani']   = dftest['content'].apply(lambda x: predict(x))

ValueError: ignored